<h1>Import Required Libraries</h1>

In [1]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# k-fold cross validation evaluation of XGBoost model
from numpy import loadtxt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# plot decision tree
from xgboost import plot_tree
from matplotlib import pyplot
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

<h1> Dataset1 </h1>

<h3>Load Data</h3>

In [7]:
df = pd.read_csv('test2.csv')
df.head()

,Id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Response,Complain
0,1826,1970,Graduation,Divorced,84835.0,0,0,6/16/2014,0,189,...,111,189,218,1,4,4,6,1,1,0
1,1,1961,Graduation,Single,57091.0,0,0,6/15/2014,0,464,...,7,0,37,1,7,3,7,5,1,0
2,10476,1958,Graduation,Married,67267.0,0,1,5/13/2014,0,134,...,15,2,30,1,3,2,5,2,0,0
3,1386,1967,Graduation,Together,32474.0,1,1,11/5/2014,0,10,...,0,0,0,1,1,0,2,7,0,0
4,5371,1989,Graduation,Single,21474.0,1,0,8/4/2014,0,6,...,11,0,34,2,3,1,2,7,1,0


In [8]:
# Dropping ID Column beacause we dont id column for predictions
df.drop('Id', axis=1, inplace = True)

In [9]:
# Check for null values in the dataset
df.isnull().sum()

Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
Response                0
Complain                0
dtype: int64

In [10]:
def fill_na(frame):
    for i in frame.columns:
        if(((frame[i].isnull().sum() / len(frame))*100) <= 30) & (frame[i].dtype == 'int64'):
            frame[i] = frame[i].fillna(frame[i].median())

        elif(((frame[i].isnull().sum() / len(frame))*100) <= 30) & (frame[i].dtype == 'O'):
            frame[i] = frame[i].fillna(frame[i].mode()[0])

        elif(((frame[i].isnull().sum() / len(frame))*100) <= 30) & (frame[i].dtype == 'float64'):
            frame[i] = frame[i].fillna(frame[i].median())

fill_na(df)

In [11]:
# Converting dt_Customer into datetime64 data type
df['Dt_Customer'] = df['Dt_Customer'].astype('datetime64')
# Creating two new columns Date_customer and Month_customer from Dt_Customer column
df['Date_Customer'] = df['Dt_Customer'].dt.day
df['Month_Customer'] = df['Dt_Customer'].dt.month
df['Year_Customer'] = df['Dt_Customer'].dt.year
# Now we can drop Dt_Customer column
df.drop('Dt_Customer', axis=1, inplace=True)

In [12]:
def encode(dataframe):
    lec = LabelEncoder()
    for j in dataframe.columns:
        if(dataframe[j].dtype == 'object'):
            dataframe[j] = lec.fit_transform(dataframe[j])

encode(df)

Split data into train and test

In [13]:
x = df.drop('Response', axis=1)
y = df['Response']

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=1)

<h2> 1. XGBoost Classifier</h2?

<h3> Sử dụng train-test set</h3>

In [14]:
model1 = XGBClassifier()
model1.fit(X_train, Y_train)
predictions = model1.predict(X_test)
accuracy = accuracy_score(Y_test, predictions)
print('Accuracy: %.2f%%' % (accuracy*100))

Accuracy: 85.86%


<h3>k-fold cross-validation</h3>

In [15]:
model1 = XGBClassifier()
kfold = KFold(n_splits=10,shuffle=True, random_state=7)
results = cross_val_score(model1, x, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 87.50% (1.56%)


<h2> 2. Gradient Boosting Classifier </h2>

<h3> Sử dụng train-test set</h3>

In [16]:
model2 = GradientBoostingClassifier()
model2.fit(X_train, Y_train)
predictions = model2.predict(X_test)
accuracy = accuracy_score(Y_test, predictions)
print('Accuracy: %.2f%%' % (accuracy*100))

Accuracy: 84.97%


<h3>k-fold cross-validation</h3>

In [17]:
# CV model
model2 = GradientBoostingClassifier()
kfold = KFold(n_splits=10,shuffle=True, random_state=7)
results = cross_val_score(model2, x, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 86.96% (1.75%)


<h2> 3. Random Forest Classifier </h2>

<h3> Sử dụng train-test set</h3>

In [18]:
model3 = RandomForestClassifier()
model3.fit(X_train, Y_train)
predictions = model3.predict(X_test)
accuracy = accuracy_score(Y_test, predictions)
print('Accuracy: %.2f%%' % (accuracy*100))

Accuracy: 85.42%


<h3>k-fold cross-validation</h3>

In [19]:
model3 = RandomForestClassifier()
kfold = KFold(n_splits=10,shuffle=True, random_state=7)
results = cross_val_score(model3, x, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 87.10% (2.65%)
